In [1]:

import sqlite3

import numpy
import pandas

from data_algebra.data_ops import *
import data_algebra.SQLite

# some example data
d = pandas.DataFrame({
    'group': ['A', 'B', None, 'A', numpy.NAN, 'C'],
    'c1': [1, 2, 3, 4, 5, 6],
    'c2': [-1, -2, -3, -4, -5, -6],
    })

d

,group,c1,c2
0,A,1,-1
1,B,2,-2
2,None,3,-3
3,A,4,-4
4,NaN,5,-5
5,C,6,-6


In [2]:
ops = describe_table(d, table_name='d'). \
    extend({'choice': "group=='A'"}). \
    extend({'choice_fixed': 'choice.is_bad().if_else(0, choice)'}). \
    extend({'rc': 'choice_fixed.if_else(c1, c2)'})

print(ops)

TableDescription(
 table_name='d',
 column_names=[
   'group', 'c1', 'c2']) .\
   extend({
    'choice': "group == 'A'"}) .\
   extend({
    'choice_fixed': 'choice.is_bad().if_else(0, choice)'}) .\
   extend({
    'rc': 'choice_fixed.if_else(c1, c2)'})


In [3]:
res_1 = ops.transform(d)
res_1

,group,c1,c2,choice,choice_fixed,rc
0,A,1,-1,True,1,1
1,B,2,-2,False,0,-2
2,None,3,-3,False,0,-3
3,A,4,-4,True,1,4
4,NaN,5,-5,False,0,-5
5,C,6,-6,False,0,-6


We can do the same thing in sql, take care of the different
compare to None semantics!

In [4]:
db_model = data_algebra.SQLite.SQLiteModel()

sql = ops.to_sql(db_model, pretty=True)

print(sql)

SELECT CASE
           WHEN "choice_fixed" THEN "c1"
           WHEN NOT "choice_fixed" THEN "c2"
           ELSE NULL
       END AS "rc",
       "c1",
       "c2",
       "group",
       "choice",
       "choice_fixed"
FROM
  (SELECT "c1",
          "c2",
          "group",
          "choice",
          CASE
              WHEN is_bad("choice") THEN 0
              WHEN NOT is_bad("choice") THEN "choice"
              ELSE NULL
          END AS "choice_fixed"
   FROM
     (SELECT "c1",
             "c2",
             "group",
             "group" = 'A' AS "choice"
      FROM "d") "extend_1") "extend_2"


In [5]:

with sqlite3.connect(':memory:') as con:
    db_model.prepare_connection(con)
    d.to_sql(name='d', con=con)
    res_db = pandas.read_sql(sql, con=con)

res_db


,rc,c1,c2,group,choice,choice_fixed
0,1,1,-1,A,1.0,1
1,-2,2,-2,B,0.0,0
2,-3,3,-3,None,NaN,0
3,4,4,-4,A,1.0,1
4,-5,5,-5,None,NaN,0
5,-6,6,-6,C,0.0,0
